### Choose the first entry of each patient. Extract all the lab data for each patient. 

**1. Extract sub-categories patient id from cardiovascular**

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# change to your folder path
# os.chdir('C:/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1')
os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/DataExtract/data') 

# import diagnosis.csv
df_diagnosis = pd.read_csv('diagnosis.csv')
df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'], inplace=True)

# select cardiovascular patients
df_cardiovascular = df_diagnosis[df_diagnosis['diagnosisstring'].str.contains('cardiovascular')]


# get shock patient 
shock_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('shock')]

# get ventricular patient 
ventricular_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('ventricular')]

# get chest pain patient 
chest_pain_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('chest pain')]

# get arrhythmias patient 
arrhythmias_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('arrhythmias')]

# put id together
df_wanted = pd.concat([shock_patient, ventricular_patient, chest_pain_patient, arrhythmias_patient])
# print(df_wanted)

# Get the patient ids from df_wanted & sort the patient id
# patient_id_all multiple entry patient's stayid
patient_id_all = df_wanted['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

[ 143870  151179  151900 ... 3351297 3352230 3352231]


**2. Exclude patient whose unitvisitnumbe>1**

In [18]:
# import patient.csv
df_patient = pd.read_csv('patient.csv')
df_patient.sort_values(by=['patientunitstayid'], inplace=True)
df_patient_buf = df_patient[df_patient['patientunitstayid'].isin(patient_id_all)]
df_1time_patient = df_patient_buf[df_patient_buf['unitvisitnumber']==1]
# print(df_1time_patient)

# select the patient id from df_1time_patient
patient_id = df_1time_patient['patientunitstayid'].unique()
print(f'Total number of patients: {len(patient_id)}')

Total number of patients: 915


**3. Extract data of patients within the id list**

In [19]:
# import lab.csv
df_lab = pd.read_csv('lab.csv')
df_lab.sort_values(by=['patientunitstayid', 'labresultoffset'], inplace=True)

# select the wanted patient
df_lab = df_lab[df_lab['patientunitstayid'].isin(patient_id)]

# print the shape of the wanted file
print(f'lab shape: {df_lab.shape}')

lab shape: (182390, 10)


**4. Extract lab results & create index**

In [20]:
def create_index(df):
    """
    Create first occurrence index for every patient
    """
    # create a dictionary to store the first occurrence index
    value_position_dict = {}
    first_occurrences = []
    for idx, value in enumerate(df['patientunitstayid']):
        # if the value is not in the dictionary, add it and create index
        if value not in value_position_dict:
            value_position_dict[value] = idx
            first_occurrences.append(idx)

    first_occurrences.append(len(df))
    # create first occurrence index for every patient
    df_index = pd.Series(first_occurrences)
    return df_index

In [21]:
# lab BUN
df_lab_BUN = df_lab[df_lab['labname']=='BUN']
df_lab_BUN = df_lab_BUN.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'BUN'})

# lab WBC x 1000
df_lab_WBC = df_lab[df_lab['labname']=='WBC x 1000']
df_lab_WBC = df_lab_WBC.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'WBC x 1000'})

# lab bicarbonate
df_lab_bicarbonate = df_lab[df_lab['labname']=='bicarbonate']
df_lab_bicarbonate = df_lab_bicarbonate.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'bicarbonate'})

# lab sodium
df_lab_sodium = df_lab[df_lab['labname']=='sodium']
df_lab_sodium = df_lab_sodium.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'sodium'})

# lab potassium
df_lab_potassium = df_lab[df_lab['labname']=='potassium']
df_lab_potassium = df_lab_potassium.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'potassium'})

# lab total bilirubin
df_lab_bilirubin = df_lab[df_lab['labname']=='total bilirubin']
df_lab_bilirubin = df_lab_bilirubin.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'total bilirubin'})

In [22]:
# extract lab results from lab.csv
BUN = df_lab_BUN[['patientunitstayid', 'observationoffset', 'BUN']].copy()
WBC = df_lab_WBC[['patientunitstayid', 'observationoffset', 'WBC x 1000']].copy()
bicarbonate = df_lab_bicarbonate[['patientunitstayid', 'observationoffset', 'bicarbonate']].copy()
sodium = df_lab_sodium[['patientunitstayid', 'observationoffset', 'sodium']].copy()
potassium = df_lab_potassium[['patientunitstayid', 'observationoffset', 'potassium']].copy()
bilirubin = df_lab_bilirubin[['patientunitstayid', 'observationoffset', 'total bilirubin']].copy()

# sort the lab results by patient id and observationoffset
BUN.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
WBC.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
bicarbonate.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
sodium.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
potassium.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
bilirubin.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)

# print the head of the lab results
# print(f'First 5 rows of lab_BUN: \n{BUN.head()} \n')
# print(f'First 5 rows of lab_WBC: \n{WBC.head()} \n')
# print(f'First 5 rows of lab_bicarbonate: \n{bicarbonate.head()} \n')
# print(f'First 5 rows of lab_sodium: \n{sodium.head()} \n')
# print(f'First 5 rows of lab_potassium: \n{potassium.head()} \n')
print(f'First 5 rows of lab_bilirubin: \n{bilirubin.head()} \n')

# create index for each variable
BUN_index = create_index(BUN)
WBC_index = create_index(WBC)
bicarbonate_index = create_index(bicarbonate)
sodium_index = create_index(sodium)
potassium_index = create_index(potassium)
bilirubin_index = create_index(bilirubin)

First 5 rows of lab_bilirubin: 
        patientunitstayid  observationoffset  total bilirubin
341828             151179               -187              0.4
341827             151179                144              0.2
76036              151900               -265              0.5
23626              152954               -143              0.3
23763              152954                494              0.4 

